In [1]:
import os
import re
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [10]:

def pretty_print(grid):
    for row in grid:
        print(row)

#from path of description file get the coupling grid, model data and 
#remove models with error
def process_file(path):
    with open(path,'r') as f:
        #don't consider first two lines
        f.readline()           #segmenti x x x
        f.readline()           #rotazioni x x x

        line=f.readline()      #numero pezzi x  

        #get number of fragments
        number=int(re.findall("\d+",line)[0])

        #coupling matrix
        grid=[]
        for i in range(number):
            line=f.readline()
            ret=re.findall('-1|0|1',line)
            grid.append(list(map(int,ret)))

        #model data
        model_names=[]
        non_valid_indeces=[]
        for m in range(number):
            f.readline()        #blank line
            name=f.readline()   #model name
            mesh=f.readline()   #mesh n
            f.readline()        #external n
            f.readline()        #internal n

            #if mesh=0 I don't consider the element
            mesh_n=int(mesh.rstrip().split(" ")[1]) 
            if mesh_n!=0: 
                #try create file name
                file_name= name.rstrip().replace(".","_")
                model_names.append(file_name)
                
            else:
                #saving indices to remove later
                non_valid_indeces.append(m)

        #removing elements from grid
        #sorted in reverse to avoid wrong index
        for index in sorted(non_valid_indeces, reverse=True):
            del grid[index]
            for row in grid:
                del row[index]
        


        #eliminare anche le colonne

        return grid,model_names


In [6]:
def get_set(folder,verbose=True):
    folder_path=os.path.join(main_path,folder)
    models_file=[]

    #for each set of pieces
    for file in os.listdir(folder_path):
        if '.txt' in file:
            description_file=file
        elif '.stl' in file:
            #not used, they are not in the same order of the file
            models_file.append(file)
    #parsing description file
    full_description_file=os.path.join(folder_path,description_file)
    grid,model_names=process_file(full_description_file)

    #get path of models of current set
    model_prefix=folder.replace("generatedTest_","")
    complete_models_path=[]
    for i in range(len(model_names)):
        #e.g. 2021_11_29_10_00_40_Cube_001.stl
        correct_model_name=f"{model_prefix}_{model_names[i]}.stl"

        #saving only names, could be useful
        model_names[i]=correct_model_name
        complete_models_path.append(os.path.join(folder_path,correct_model_name))

    #create set and put into list of sets
    set={"models":complete_models_path,"grid":grid}
    if verbose:
        print("A set: \n")
        print("Models: ",set["models"])
        print("Grid: ")
        pretty_print(set["grid"])

    return set

In [7]:
#here i save all the sets of fragments
sets=[]

#root folder
main_path="produzione_29112021"

In [14]:

for folder in os.listdir(main_path):
    #check only folders, not files
    if '.' not in folder:
        set=get_set(folder)
        sets.append(set)
        break       

A set: 

Models:  ['produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_001.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_002.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_003.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_004.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_005.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_006.stl', 'produzione_29112021\\generatedTest_2021_11_29_10_00_40\\2021_11_29_10_00_40_Cube_007.stl']
Grid: 
[-1, 1, 0, 0, 1, 1, 1, 1]
[1, -1, 1, 0, 1, 1, 1, 0]
[0, 1, -1, 1, 1, 1, 1, 0]
[0, 0, 1, -1, 1, 0, 1, 0]
[1, 1, 1, 1, -1, 0, 1, 1]
[1, 1, 1, 0, 0, -1, 1, 0]
[1, 1, 1, 1, 1, 1, -1, 1]
[1, 0, 0, 0, 1, 0, 1, -1]


## Visualize 3D models and pointCloud

In [15]:
#visualize some models and point clouds
for set in sets:
    for path in set["models"]:
        mesh = o3d.io.read_triangle_mesh(path)
        pointcloud = mesh.sample_points_poisson_disk(1000)

        # you can plot and check
        o3d.visualization.draw_geometries([mesh])
        o3d.visualization.draw_geometries([pointcloud])
        
        break

KeyboardInterrupt: 